# SCS Python Interface Tutorial

This tutorial only covers the specifics of the SCS Python interface. For background on SCS and descriptions of the optimization problem being solved and the input data format, please see the [`SCS README`](https://github.com/cvxgrp/scs) or the [SCS Paper](http://web.stanford.edu/~boyd/papers/scs.html).

This tutorial covers **most** (but not all) of the Python interface. For more details, see the [`SCS_Python README`](https://github.com/ajfriend/scs_python).

# Basic SCS Interface

In [1]:
import scs

We'll load data for an example $\ell_1$ minimization problem. `data` and `cone` are correctly-formatted input for the SCS solver.

In [2]:
m = 2000
data, cone = scs.examples.l1(m=m)
print 'data: ', data
print 'cone: ', cone

data:  {'A': <10000x8000 sparse matrix of type '<type 'numpy.float64'>'
	with 816000 stored elements in Compressed Sparse Column format>, 'c': array([ 0.,  0.,  0., ...,  1.,  1.,  1.]), 'b': array([-1.06370562,  1.2659559 ,  0.0602342 , ...,  0.        ,
        0.        ,  0.        ])}
cone:  {'l': 8000, 'f': 2000}


- Use `scs.solve()` to compute a solution to the problem
- `data` and `cone` are required arguments
- Solver settings can be passed as keyword arguments
- `result` is a dictionary with keys `'x'`, `'y'`, `'s'` (`numpy` arrays corresponding to problem variables), and `'info'` (a `dict` giving solver status information)

In [3]:
%%time
result = scs.solve(data, cone, verbose=False)
print('---###---')

---###---
CPU times: user 9.87 s, sys: 159 ms, total: 10 s
Wall time: 10.3 s


In [4]:
print result.keys()
result['info']

['y', 'x', 's', 'info']


{'dobj': 236.77520372459114,
 'iter': 540,
 'pobj': 236.775714014689,
 'relGap': 1.075311581486246e-06,
 'resDual': 0.0009946001367409107,
 'resInfeas': 11.836049744716343,
 'resPri': 0.0006893041237640758,
 'resUnbdd': nan,
 'setupTime': 4610.10807,
 'solveTime': 5645.831647,
 'status': u'Solved',
 'statusVal': 1}

A **copy** of the SCS default solver settings can be seen by calling `scs.default_settings()`. Each of these settings can be modified for a call to `scs.solve()` by passing in keyword arguments.

In [5]:
scs.default_settings()

{'alpha': 1.5,
 'cg_rate': 2.0,
 'eps': 0.001,
 'max_iters': 2500,
 'normalize': True,
 'rho_x': 0.001,
 'scale': 1.0,
 'use_indirect': False,
 'verbose': True,
 'warm_start': False}

# Warm-Starting

The solver can be warm-started with vectors `x`, `y`, and `s` (which are expected to be close to the solution). This can reduce the number of iterations needed to converge.

Pass a dictionary with keys `'x'`, `'y'`, and `'s'` (pointing to `numpy` arrays of the appropriate size) to the `sol` keyword argument of `scs.solve()`, and set the option `warm_start=True`. Note that all three vectors are needed to warm-start.

When `scs.solve()` is called without the `sol` argument, the function creates `numpy` arrays automatically and returns them in `result`.

Below, we use the previous solution to warm-start the solver on the same problem, and see that this allows the solver to exit after 0 iterations.

In [6]:
%%time
sol = dict(x=result['x'],y=result['y'],s=result['s'])
result = scs.solve(data, cone, sol=sol, verbose=True, warm_start=True)
print('---###---')

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 816000
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 8000, constraints m = 10000
Cones:	primal zero / dual free vars: 2000
	linear vars: 8000
Setup time: 4.63e+00s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.86e-04  9.92e-04  1.03e-06  2.37e+02  2.37e+02  0.00e+00  3.31e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 3.34e-02s
	Lin-sys: nnz in L factor: 2837000, avg solve 

We can confirm the number of iterations by inspecting the `'info'` dictionary. Note that even though
the solver needed 0 iterations to converge, it still had to perform a matrix factorization (since `use_indirect=False`), which took about 4 seconds.

In [8]:
result['info']

{'dobj': 236.77420540751393,
 'iter': 0,
 'pobj': 236.77469235908313,
 'relGap': 1.0261357080219337e-06,
 'resDual': 0.0009920501135128545,
 'resInfeas': 11.836119032312665,
 'resPri': 0.0006857237060205455,
 'resUnbdd': nan,
 'setupTime': 4627.85207,
 'solveTime': 33.368644,
 'status': u'Solved',
 'statusVal': 1}

# Factorization Caching Interface

When using the direct method (`use_indirect=False`), we can cache the matrix factorization involving `A`, and reuse it across several solves. This is useful when solving a sequence or family of problems where `A` is fixed, but `b` and `c` may change.

The `scs.Workspace` object caches the matrix factorization for us, and allows us to call the solver many times with different values for `b` and `c`. We can also optionally warm-start the solver, and change **some** of the solver settings between solves.

Below, we initialize the `Workspace` object with the same data as above, and note that the setup time (factorization time) is still approximately 4 seconds. Note that the `Workspace` defaults to the direct (factorization) method because `use_indirect=False`, unless the user specifies otherwise.

In [9]:
%%time
work = scs.Workspace(data, cone)
print('---###---')

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 816000
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 8000, constraints m = 10000
Cones:	primal zero / dual free vars: 2000
	linear vars: 8000
Setup time: 3.99e+00s
---###---
CPU times: user 3.9 s, sys: 52.7 ms, total: 3.95 s
Wall time: 3.99 s


## `work.info`

`work.info` will give a **copy** of the `info` dictionary, showing solver status information. Since only the setup has run, only the `setupTime` key is nonzero.

In [10]:
work.info

{'dobj': 0.0,
 'iter': 0,
 'pobj': 0.0,
 'relGap': 0.0,
 'resDual': 0.0,
 'resInfeas': 0.0,
 'resPri': 0.0,
 'resUnbdd': 0.0,
 'setupTime': 3985.002903,
 'solveTime': 0.0,
 'status': u'',
 'statusVal': 0}

## `work.settings`

The `Workspace` object records changes to the user-specified settings. They can be inspected and modified through the
`work.settings` dictionary. When the solver is run, it will operate based on the current `settings`.

Some settings should not be changed once the `Workspace` object is initialized:
- `use_indirect`
- `normalize`
- `scale`
- `rho_x`

Please do not modify these settings after the `Workspace` has been initialized. Changing them will give incorrect results, since the cached matrix factorization depends on them.

In [11]:
work.settings

{'alpha': 1.5,
 'cg_rate': 2.0,
 'eps': 0.001,
 'max_iters': 2500,
 'normalize': True,
 'rho_x': 0.001,
 'scale': 1.0,
 'use_indirect': False,
 'verbose': True,
 'warm_start': False}

## `work.fixed`

The user can see a **copy** of the settings fixed at `Workspace` initialization time with the `work.fixed` attribute.
The `Workspace` will raise an exception if these values are changed in `settings`.

In [12]:
work.fixed

{'normalize': True, 'rho_x': 0.001, 'scale': 1.0, 'use_indirect': False}

## `work.data`

The vectors `b` and `c` can be modified through the `work.data` dictionary. Please do not modify the matrix `A` after the `Workspace` has been initialized. Changing it will give incorrect results, since the cached matrix factorization depends on the original `A`.

In [13]:
work.data

{'A': <10000x8000 sparse matrix of type '<type 'numpy.float64'>'
 	with 816000 stored elements in Compressed Sparse Column format>,
 'b': array([-1.06370562,  1.2659559 ,  0.0602342 , ...,  0.        ,
         0.        ,  0.        ]),
 'c': array([ 0.,  0.,  0., ...,  1.,  1.,  1.])}

## `work.sol`
The solution vectors `x`, `y`, and `s` are stored in `work.sol`. These are the `numpy` arrays where the solution will be written. The user is free to modify this dictionary and/or the vectors themselves, but do ensure the sizes of the vectors remain unchanged.

The user may change these vectors at any time to write the solution to a different location, or to input warm-starting vectors.

By default, the `Workspace()` constructor will create empty `numpy` arrays to populate `work.sol`. The user can supply their own by passing a dictionary to the `sol` keyword argument of `Workspace()`.

In [14]:
work.sol

{'s': array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 'x': array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 'y': array([ 0.,  0.,  0., ...,  0.,  0.,  0.])}

## `work.solve()`

`work.solve()` will call the solver based on the current `settings`, `sol` (if `warm_start=True`), `data['b']` and `data['c']`.

Calling the solver below, note that we skip the setup step, and go right to the iterative procedure.

In [15]:
%%time
result = work.solve()
print('---###---')

----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.57e-02 
   100| 7.99e-03  1.05e-02  3.04e-05  2.37e+02  2.37e+02  1.13e-14  1.23e+00 
   200| 3.13e-03  4.02e-03  7.04e-06  2.37e+02  2.37e+02  1.13e-14  2.32e+00 
   300| 1.77e-03  2.30e-03  1.38e-06  2.37e+02  2.37e+02  1.13e-14  3.37e+00 
   400| 1.11e-03  1.53e-03  2.07e-06  2.37e+02  2.37e+02  1.13e-14  4.44e+00 
   500| 7.85e-04  1.09e-03  5.64e-07  2.37e+02  2.37e+02  1.13e-14  5.49e+00 
   540| 6.89e-04  9.95e-04  1.08e-06  2.37e+02  2.37e+02  1.13e-14  5.92e+00 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 5.92e+00s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 1.05e-02s
	Cones: avg projection time: 3.48e-05s


Note that the solution is stored in the `work` object in `work.sol`.

In [16]:
work.sol

{'s': array([  5.71284582e-17,  -1.17107198e-16,   1.10606304e-16, ...,
         -7.06831992e-18,   5.30123994e-18,   2.41232642e-01]),
 'x': array([  7.10379611e-05,   2.99788766e-05,  -1.08941016e-05, ...,
         -4.98375019e-08,  -4.98375019e-08,   1.20615895e-01]),
 'y': array([ 0.19930045, -0.21738546,  0.24355537, ...,  0.89722381,
         0.7904324 ,  0.        ])}

## Workspace `settings`

By default, the `Workspace` will use the settings from `scs.default_settings()`. The user has a few opportunities to modify them:

- at initialization time, by passing keyword arguments to `scs.Workspace()`
- between solves, by modifying `work.settings` (but making sure **not** to modify the settings in `work.fixed`)
- just before solve time, by passing keyword arguments to `work.solve()`

Any changes to the `settings` persist in the `Workspace` object, including those passed to `scs.solve()`. For instance,

```
work.solve(eps=1e-5, alpha=1.1)
```

is exactly equivalent to

```
work.settings['eps'] = 1e-5
work.settings['alpha'] = 1.1
work.solve()
```

## Caching and warm-starting

In addition to benefiting from the cached matrix factorization, we can also use a warm-started solution by calling
`work.solve(warm_start=True)`. This will warm-start from the vectors in `work.sol`. Since we previously ran the solver and the solution is already contained in `work.sol`, this should require 0 iterations. It should also require 0 setup time, since we've cached the factorization.

In [17]:
%%time
result = work.solve(warm_start=True)
print('---###---')

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.86e-04  9.92e-04  1.03e-06  2.37e+02  2.37e+02  0.00e+00  3.39e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 3.41e-02s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 2.56e-02s
	Cones: avg projection time: 9.24e-05s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 5.5713e-16, dist(y, K*) = 0.0000e+00, s'y/m = 5.2014e-20
|Ax + s - b|_2 / (1 + |b|_2) = 6.8572e-04
|A'y + c|_2 / (1 + |c|_2) = 9.9205e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 1.0261e-06
----------------------------------------------------------------------------
c'x = 236.7747, -b'y = 236.7742
---###---
CPU times: user 34.7 ms, s

## Perturbed warm-starting

For a more interesting example of warm-starting, we perturb the `b` vector a bit and try to re-solve.
However, if we warm-start from the previous solution to the unperturbed problem, we can expect to only need a few iterations to "correct" for the perturbation and obtain the new solution.

In [18]:
%%time
# make a small perturbation to b
work.data['b'][:m] += .01
result = work.solve(warm_start=True)
print('---###---')

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 7.63e-04  1.22e-03  3.53e-06  2.37e+02  2.37e+02  0.00e+00  3.45e-02 
    20| 6.89e-04  9.90e-04  3.63e-06  2.37e+02  2.37e+02  3.02e-14  2.68e-01 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.68e-01s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 1.20e-02s
	Cones: avg projection time: 3.96e-05s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 5.4599e-16, dist(y, K*) = 0.0000e+00, s'y/m = -8.3117e-20
|Ax + s - b|_2 / (1 + |b|_2) = 6.8888e-04
|A'y + c|_2 / (1 + |c|_2) = 9.8963e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 3.6280e-06
------------------------------------------------------------------

If we attempt to solve the problem without warm-starting, it will require many more iterations (and a longer solve time).

In [19]:
%%time
result = work.solve(warm_start=False)
print('---###---')

----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.41e-02 
   100| 7.98e-03  1.05e-02  2.65e-05  2.37e+02  2.37e+02  2.26e-14  1.19e+00 
   200| 3.13e-03  4.01e-03  9.53e-06  2.37e+02  2.37e+02  2.26e-14  2.24e+00 
   300| 1.77e-03  2.30e-03  1.75e-06  2.37e+02  2.37e+02  2.26e-14  3.29e+00 
   400| 1.11e-03  1.53e-03  1.21e-06  2.37e+02  2.37e+02  2.26e-14  4.34e+00 
   500| 7.54e-04  1.12e-03  1.79e-07  2.37e+02  2.37e+02  2.26e-14  5.39e+00 
   540| 6.97e-04  9.78e-04  1.54e-06  2.37e+02  2.37e+02  2.26e-14  5.82e+00 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 5.82e+00s
	Lin-sys: nnz in L factor: 2837000, avg solve time: 1.04e-02s
	Cones: avg projection time: 3.40e-05s


If we revert even futher (back to the point where we started this tutorial) and try to solve **without** factorization caching or warm-starting, we can expect an even longer solve time.

In [20]:
%%time
data = work.data
result = scs.solve(data, cone, warm_start=False)
print('---###---')

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 816000
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 8000, constraints m = 10000
Cones:	primal zero / dual free vars: 2000
	linear vars: 8000
Setup time: 3.99e+00s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  3.47e-02 
   100| 7.98e-03  1.05e-02  2.65e-05  2.37e+02  2.37e+02  2.26e-14  1.10e+00 
   200| 3.13e-03  4.01e-03  9.53e-06  2.37e+02  2.37e+02  2.26e-14  2.57e+00 
   300| 1.77e-03  2.30e-03  1.75e-06  2.37e+0